Naive sampling method.
This method is wrong, and will give unpleasant results.

In [1]:
import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
# convert graph to a dictionary, with node as key, and its neighbours (list) as values
graphDict = {}

# helper variables for constructing a crs matrix
node = []
neighbours = []
edges = []

# read file and process data
with open("train.txt", 'r') as f:
    for data in f:
        converted_data = list(map(int, data.split()))
        source = converted_data[0]
        sink = converted_data[1:]
        
        graphDict[source] = sink
        for s in sink:
            node.append(source)
            neighbours.append(s)
            edges.append(1)

In [3]:
print(max(node), max(neighbours))

4867036 4867135


In [4]:
sources = set(node)
sinks = [node for node in neighbours if node not in sources]
sources = list(sources) # do not convert to list at first, b.c. set is faster when checking existence of an element
print(len(sources), len(sinks))

19570 22200180


In [5]:
from scipy.sparse import csr_matrix

_max = max(neighbours) + 1
source_sink_matrix=csr_matrix((edges,(node,neighbours)),shape=(_max, _max))

In [6]:
import random

max_sample = 50000
random.seed(100)

def generate_2_random_numbers(_range):
    return random.choice(range(_range)), random.choice(range(_range))

def generate_positive_samples(_range):
    sample = set()
    for node in sources:
        target = random.choice(graphDict[node])
        sample.add((node, target)) # only 1 neighbour per node, will this cause an issue?
    return list(sample)

def generate_negative_samples(_range):
    # should I randomly generate source and sink
    # or should I use the source set and generate random sink only?
    sample = set()
    for node in sources:
        target = random.choice(sinks)
        sample.add((node, target)) # only 1 neighbour per node, will this cause an issue?
    return list(sample)

In [7]:
pos_samples = generate_positive_samples(max_sample)
neg_samples = generate_negative_samples(max_sample)
print(len(pos_samples), len(neg_samples))

19570 19570


In [8]:
node_1 = []
node_2 = []
edge = []
for x1, x2 in pos_samples:
    node_1.append(x1)
    node_2.append(x2)
    edge.append(1)
for x1, x2 in neg_samples:
    node_1.append(x1)
    node_2.append(x2)
    edge.append(0)
    
train_df = pd.DataFrame({"node_1": node_1, "node_2": node_2, "edge": edge})

In [9]:
G = nx.Graph()
G.add_edges_from(pos_samples)

In [10]:
from node2vec import Node2Vec

n2v = Node2Vec(G, dimensions=100, walk_length=16, num_walks=50)

Generating walks (CPU: 1): 100%|██████████| 50/50 [13:20<00:00, 16.01s/it]


In [11]:
n2v_model = n2v.fit(window=7, min_count=1)
train_embeddings = [n2v_model[(str(i), str(j))] for i, j in pos_samples] + [n2v_model[(str(i), str(j))] for i, j in neg_samples]

/Users/jiachengye/opt/anaconda3/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


KeyboardInterrupt: 

In [ ]:
x_train, y_train = train_test_split(np.array(train_embeddings), np.array(edge), test_size=1, random_state=95)
lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)

In [ ]:
test_samples = generate_positive_samples(2000) + generate_negative_samples(2000)
test_y = [1 for i in range(2000)] + [0 for i in range(2000)]
print(len(test_samples))

test_embeddings = [n2v_model[pair] for pair in test_samples]
predicted_proba = lr_clf.predict_proba(test_embeddings)
predicted_label = [p[1] for p in predicted_proba]
auc = roc_auc_score(test_y, predicted_label)
print(auc)

In [ ]:
test_pair = []
with open("test-public.txt", "r") as f:
    next(f)
    for data in f:
        line = data.split()
        test_pair.append((line[1], line[2]))
print(len(test_pair))

test_embeddings = [n2v_model[pair] for pair in test_pair]
predicted = lr_clf.predict_proba(test_embeddings)

In [ ]:
import csv
with open("predict_yjc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Id", "Predicted"])
    _id = 1
    for p in predicted:
        writer.writerow([_id, p])
        _id += 1